# 自前のネットワークを組んで学習させてみる

In [2]:
import pandas as pd
import numpy as np
import os
import pickle
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
plt.rcParams['font.family'] = 'Meiryo'

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models 
import pprint

In [3]:
train_fn = r'./dataset_cur_train.csv'
test_fn = r'./dataset_cur_test.csv'

In [4]:
df_train = pd.read_csv(train_fn,index_col=0)
df_train['red_diff'] = df_train['red_diff'].astype(np.float32)
df_train['remain_ends'] = df_train['remain_ends'].astype(np.float32)
df_train['last_stone_is_red'] = df_train['last_stone_is_red'].astype(np.float32)
df_train['red_postion'] = df_train['red_postion'].astype(np.float32)
display(df_train.dtypes)
display(df_train.head(3))

red_diff             float32
remain_ends          float32
last_stone_is_red    float32
red_postion          float32
filepath              object
dtype: object

,red_diff,remain_ends,last_stone_is_red,red_postion,filepath
8,1.0,0.0,1.0,3.0,./dataset_o\ECC2023_ResultsBook_Women_A-Divisi...
0,2.0,8.0,1.0,0.0,./dataset_o\ECC2021_ResultsBook_Men_A-Division...
8,-2.0,0.0,0.0,3.0,./dataset_o\ECC2019_ResultsBook_Women_A-Divisi...


## red_diff が目的、fn,remain_ends,last_stone_is_red,red_postion が説明変数
説明変数を標準化する

In [5]:
# 標準化
stdsc = StandardScaler()
##学習時の標準化したパラメータは、評価、本番時におなじ重みで標準化する処理が必要
x_train_df = df_train.copy().drop(['filepath','red_diff'],axis=1)
x_train_std = stdsc.fit_transform(x_train_df)
display( x_train_std[:3] )
## DataFrameの値を入れ替え
qcl = df_train.columns.to_list()
qcl.remove('filepath')
qcl.remove('red_diff')
print(qcl)
df_train[qcl] = x_train_std
pickle.dump(stdsc, open("stdsc_02240210.pkl", "wb"))
display(stdsc.n_features_in_, stdsc.mean_ , stdsc.var_) 
df_train.head(5)

array([[-1.5558919 ,  0.9972611 ,  1.2323233 ],
       [ 1.5270197 ,  0.9972611 , -0.01566701],
       [-1.5558919 , -1.0027465 ,  1.2323233 ]], dtype=float32)

['remain_ends', 'last_stone_is_red', 'red_postion']


3

array([4.0374607 , 0.50137133, 0.03766138])

array([6.73376293, 0.24999812, 5.7785669 ])

,red_diff,remain_ends,last_stone_is_red,red_postion,filepath
8,1.0,-1.555892,0.997261,1.232323,./dataset_o\ECC2023_ResultsBook_Women_A-Divisi...
0,2.0,1.527020,0.997261,-0.015667,./dataset_o\ECC2021_ResultsBook_Men_A-Division...
8,-2.0,-1.555892,-1.002746,1.232323,./dataset_o\ECC2019_ResultsBook_Women_A-Divisi...
7,1.0,-1.170528,0.997261,-0.431664,./dataset_o\OWG2018_ResultsBook\geme835end8.png
8,-3.0,-1.555892,-1.002746,0.400330,./dataset_o\WWCC2018_ResultsBook\geme338end9.png


https://qiita.com/illumination-k/items/fa7508127d8942c1284f

In [6]:
import torch
from torchvision import transforms
import torchvision

#https://pytorch.org/vision/main/models/generated/torchvision.models.efficientnet_v2_s.html

mymean = [0.485, 0.456, 0.406]
mtstd = [0.229, 0.224, 0.225]
transforms.Normalize(mymean, mtstd)



Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])

In [7]:
## 画像変換の定義
w,h = Image.open(df_train['filepath'].values[0]).size
## 正方形にするための差分
pad = (h-w)//2
display([w,h],pad,[w+pad*2,h])
target_size = 384
## train用
transform_train = transforms.Compose([
    transforms.Pad(( pad, 0), fill=0, padding_mode='constant'),  # 左右に余白を追加
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.RandomHorizontalFlip(0.33),
    transforms.ToTensor(),
    transforms.Normalize(mymean, mtstd)
])
# valid/test用
transform_test = transforms.Compose([
    transforms.Pad(( pad, 0), fill=0, padding_mode='constant'),  # 左右に余白を追加
    transforms.Resize(target_size),
    transforms.CenterCrop(target_size),
    transforms.RandomHorizontalFlip(0.33),
    transforms.ToTensor(),
    transforms.Normalize(mymean, mtstd)
])

[300, 540]

120

[540, 540]

In [8]:
from torch.utils.data import DataLoader, Dataset

# 自前のデータセット定義

In [9]:
class ImgValueDataset(Dataset):
    def __init__(self, df, classcol , fncol , transform):
        
        ##self.label_list  = df[classcol].to_list()
        self.label_list  = pd.get_dummies(df[classcol]).values
        self.img_pathlist  = df[fncol].to_list()
        cols = df.columns.to_list()
        cols.remove(fncol)
        cols.remove(classcol)
        self.x_values = df[cols].values
        self.transform = transform

    def __len__(self):  
        return len( self.img_pathlist )
    
    def __getitem__(self, index):
        # 画像をPILとして読み込む
        #print(index)
        image = Image.open(self.img_pathlist[index])
        image = image.convert("RGB") 
        
        label = self.label_list[index]
        extend = self.x_values[index] 
        if self.transform is not None:
            ##print('use transform')
            image = self.transform(image)
        return image, label 
    pass

In [10]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df_train, test_size=0.2, stratify=df_train['red_diff'])
train_dataset = ImgValueDataset( train_df ,classcol='red_diff' , fncol='filepath',transform=transform_train)
test_dataset = ImgValueDataset( test_df ,classcol='red_diff' , fncol='filepath' ,transform=transform_test)

In [11]:
batch_size = 4
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)


In [12]:
##q = Image.open(train_dataset.img_pathlist[0])
##transforms.Pad(( pad, 0), fill=0, padding_mode='constant')(q)

In [13]:
images, labels = next(iter(train_loader))
images.shape , labels

(torch.Size([4, 3, 384, 384]),
 tensor([[False, False, False, False,  True, False, False, False, False, False,
          False],
         [False, False, False, False, False, False,  True, False, False, False,
          False],
         [False, False, False, False,  True, False, False, False, False, False,
          False],
         [False, False, False, False,  True, False, False, False, False, False,
          False]]))

In [14]:
len(train_df['red_diff'])

11959

In [15]:
class_list = train_df['red_diff'].unique()
num_classes = len(class_list) 
class_list , len(class_list)

(array([-2.,  0.,  1., -1.,  2., -3.,  3., -4., -5.,  4.,  5.],
       dtype=float32),
 11)

## 転移学習用のネットを組む

In [21]:
weights = models.EfficientNet_V2_S_Weights.IMAGENET1K_V1

dropout = 0.2
inverted_residual_setting , last_channel = \
    models.efficientnet._efficientnet_conf('efficientnet_v2_s',width_mult=1.0, depth_mult=1.0)

In [37]:
weights = models.EfficientNet_V2_S_Weights.verify(weights)

In [22]:
class myEfficientNet(models.EfficientNet):
    def _forward_impl(self, x: torch.Tensor) -> torch.Tensor:
        print(x.shape)
        ## -- ここで　４枚目のレイヤを分離
        x = self.features(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        ## -- ここで結合
        x = self.classifier(x)
        return x
    pass



0.2

In [57]:
from functools import partial
from typing import Any, Callable, Dict, List, Optional, Sequence, Tuple, Union
def _efficientnet(
    inverted_residual_setting,
    dropout,
    last_channel,
    weights,
    progress,
    **kwargs,
) -> models.EfficientNet:
    #if weights is not None:
    #    _ovewrite_named_param(kwargs, "num_classes", len(weights.meta["categories"]))
    print(type(kwargs))
    model = myEfficientNet(inverted_residual_setting, dropout, last_channel=last_channel, **kwargs)

    if weights is not None:
        print('x')
        model.load_state_dict(weights.get_state_dict(progress=progress, check_hash=True))

    return model
_efficientnet(
    inverted_residual_setting,
    0.2,
    last_channel,
    weights,
    True,
    norm_layer=partial(nn.BatchNorm2d, eps=1e-03),
    num_classes=1000
)

<class 'dict'>
x


myEfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)


In [58]:
net = _efficientnet(
    inverted_residual_setting,
    0.2,
    last_channel,
    weights,
    True,
    norm_layer=partial(nn.BatchNorm2d, eps=1e-03),
    num_classes=1000
)

## 全結合層乗っ取り作戦
#lastconv_output_channels = inverted_residual_setting[-1].out_channels
#net.classifier = nn.Linear(in_features=1280, out_features=11, bias=True)

net

<class 'dict'>
x


myEfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): FusedMBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(24, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (1): BatchNorm2d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)


## 損失関数とオプティマイザ

In [59]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, verbose=True)

## 訓練（ネットワークが通るか確認）

In [60]:
from torch.autograd import Variable
def saveModel():
    path = "./20240209.Model.pth"
    torch.save(model.state_dict(), path)
    
def testAccuracy(model,test_loader):
    
    model.eval()
    accuracy = 0.0
    total = 0.0
    
    with torch.no_grad():
        for data in test_loader:
            images, labels = data
            # run the model on the test set to predict labels
            outputs = model(images)
            # the label with the highest energy will be our prediction
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            accuracy += (predicted == labels).sum().item()
    
    # compute the accuracy over all test images
    accuracy = (100 * accuracy / total)
    return(accuracy)

## 学習可能か確認

In [61]:
len(train_dataset)/64

186.859375

In [62]:
%%time
num_epochs = 2
best_accuracy = 0.0
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0,drop_last = True)
test_loader = DataLoader(test_df, batch_size=batch_size, shuffle=True, num_workers=0)

# Define your execution device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("The model will be running on", device, "device")
net = net.to(device)
min_loss = 999999999
for epoch in range(num_epochs):  # loop over the dataset multiple times
    train_loss = 0.0;val_loss = 0.0
    train_batches = 0;val_batches = 0
    running_loss = 0.0 
    net.train()  # 訓練モード
    for index_minbatch , (inputs, labels) in enumerate(train_loader, 0):
        #print(f'min_batch: start:{index_minbatch}')
        inputs ,labels_float  = inputs.to(device) , labels.float().to(device)
        # 勾配のリセット
        optimizer.zero_grad()
        
        outputs = net(inputs)    # 順方向計算
        loss = criterion(outputs, labels_float)   # 損失の計算
        loss.backward()                     # 逆方向計算(勾配計算)
        optimizer.step()                    # パラメータの更新  

        # 損失関数の変化を20ミニバッチごとに表示
        running_loss += loss.item()     # extract the loss value
        if index_minbatch % 20 == 19:    
            # print every 1000 (twice per epoch) 
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, index_minbatch + 1, running_loss / 10))
            # zero the loss
            running_loss = 0.0
            pass
    #########################################
    accuracy = testAccuracy(net,test_loader)
    print('For epoch', epoch+1,'the test accuracy over the whole test set is %d %%' % (accuracy))
    # we want to save the model if the accuracy is the best
    if accuracy > best_accuracy:
        saveModel()
        best_accuracy = accuracy

print('Finished Training')

The model will be running on cuda:0 device
torch.Size([64, 3, 384, 384])


OutOfMemoryError: CUDA out of memory. Tried to allocate 72.00 MiB. GPU 0 has a total capacty of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.70 GiB is allocated by PyTorch, and 43.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [130]:
outputs.dtype , labels_float.dtype ,labels.dtype

NameError: name 'outputs' is not defined

In [ ]:
outputs.shape , labels_float.shape

In [ ]:
len(train_loader)

In [ ]:
labels_float